In [154]:
# import libraries 

import pandas as pd 
import numpy as np
from sklearn import preprocessing
import re

In [159]:
# import dataset 
tweets = pd.read_csv('pre_tweets_keywordsfrom_2020-04-24_to_2020-05-24_#BlackLivesMatter.csv')
#tweets = pd.read_csv('pre_tweets_keywordsfrom_2020-04-24_to_2020-05-24_#BlueLivesMatter.csv')
#tweets = pd.read_csv('during_tweets_keywordsfrom_2020-05-25_to_2020-06-08_#BlackLivesMatter.csv')
#tweets = pd.read_csv('during_tweets_keywordsfrom_2020-05-25_to_2020-06-08_#BlueLivesMatter.csv')
#tweets = pd.read_csv('post_tweets_keywordsfrom_2020-06-09_to_2020-06-22_#BlackLivesMatter.csv')
#tweets = pd.read_csv('post_tweets_keywordsfrom_2020-06-09_to_2020-06-22_#BlueLivesMatter.csv')

In [160]:
tweets.head()

,Datetime,Tweet Id,Text,Username,user_mention,followersCount,listedCount
0,2020-05-23 23:58:44+00:00,1264345049782824960,#BlackTwitter #blacklivesmatter \n\nExonerated...,HaroldWNelson,NaN,16602,781
1,2020-05-23 23:57:21+00:00,1264344701915680768,#BlackTwitter #blacklivesmatter \n\nWATCH: Aft...,HaroldWNelson,NaN,16602,781
2,2020-05-23 23:57:15+00:00,1264344673650425858,This is why #poc are angry! #blacklivesmatter...,ScribeShelly,NaN,1971,55
3,2020-05-23 23:56:31+00:00,1264344489658810368,TRANSITION TO GOLFING #Biden2020 #BlackLivesM...,BuddyWinston,NaN,19437,41
4,2020-05-23 23:56:24+00:00,1264344459723935744,#BlackTwitter\n#BlackLivesMatter\n\nArchie Wil...,HaroldWNelson,theadvocatebr,16602,781


In [161]:
tweets=tweets.drop(columns=["Datetime","Tweet Id","listedCount"])

In [162]:
tweets

,Text,Username,user_mention,followersCount
0,#BlackTwitter #blacklivesmatter \n\nExonerated...,HaroldWNelson,NaN,16602
1,#BlackTwitter #blacklivesmatter \n\nWATCH: Aft...,HaroldWNelson,NaN,16602
2,This is why #poc are angry! #blacklivesmatter...,ScribeShelly,NaN,1971
3,TRANSITION TO GOLFING #Biden2020 #BlackLivesM...,BuddyWinston,NaN,19437
4,#BlackTwitter\n#BlackLivesMatter\n\nArchie Wil...,HaroldWNelson,theadvocatebr,16602
...,...,...,...,...
4995,Breonna Taylor deserved a life. Respect. Love....,robinskyleigh,NaN,5629
4996,What's this I'm reading?!\nNYPD is being parti...,HWreckz,"ACLU, splcenter",100
4997,@NPR What's this I'm reading?!\nNYPD is being ...,HWreckz,"NPR, ACLU, splcenter",100
4998,Justice for #BreyonnaTaylor! Her story needs t...,MszLisaLisa,NaN,3638


In [163]:
followerscount=tweets[["Username","followersCount"]].drop_duplicates()

In [164]:
followerscount

,Username,followersCount
0,HaroldWNelson,16602
2,ScribeShelly,1971
3,BuddyWinston,19437
5,LeBaronLordKing,163
7,NobleJennJenn,1115
...,...,...
4993,Rwillz4,1169
4994,ChukaOttih,323
4996,HWreckz,100
4998,MszLisaLisa,3638


In [165]:
# creating network 

dictionnary = []

for row in tweets.iterrows():
    if '@' not in row[1]['Text']:
        dictionnary.append([row[1][1],row[1][1]])
    else:
        for i in re.findall(r"@(\w+)",row[1]['Text']):
            dictionnary.append([row[1][1],i])
    
                

In [166]:
# formatting network 
network = pd.DataFrame(dictionnary, columns = ['source', 'target'])
network['tweet'] = network.source==network.target
network['content'] = network['tweet'].apply(lambda x: 'tweet' if x == 1 else 'non tweet')
network.drop('tweet',1, inplace=True)


C:\Users\crist\AppData\Local\Temp/ipykernel_14796/4006122528.py:5: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  network.drop('tweet',1, inplace=True)


In [167]:
network

,source,target,content
0,HaroldWNelson,HaroldWNelson,tweet
1,HaroldWNelson,HaroldWNelson,tweet
2,ScribeShelly,ScribeShelly,tweet
3,BuddyWinston,BuddyWinston,tweet
4,HaroldWNelson,theadvocatebr,non tweet
...,...,...,...
7554,HWreckz,NPR,non tweet
7555,HWreckz,ACLU,non tweet
7556,HWreckz,splcenter,non tweet
7557,MszLisaLisa,MszLisaLisa,tweet


In [168]:
network2=pd.merge(network, followerscount, how="left",left_on="source", right_on="Username")

In [169]:
network2=network2.drop(columns="Username")
network2

,source,target,content,followersCount
0,HaroldWNelson,HaroldWNelson,tweet,16602
1,HaroldWNelson,HaroldWNelson,tweet,16602
2,ScribeShelly,ScribeShelly,tweet,1971
3,BuddyWinston,BuddyWinston,tweet,19437
4,HaroldWNelson,theadvocatebr,non tweet,16602
...,...,...,...,...
7554,HWreckz,NPR,non tweet,100
7555,HWreckz,ACLU,non tweet,100
7556,HWreckz,splcenter,non tweet,100
7557,MszLisaLisa,MszLisaLisa,tweet,3638


In [170]:
network2["followersCount"].isnull().sum()

0

In [171]:
# filtering missing users that have been mentionned but no information (followers,listed) was scraped 
unique_users_network = np.unique(np.array(list(network.source) + list(network.target)))
unique_users_tweets = np.unique(np.array(tweets.Username))

In [172]:
unique_users_network

array(['00001Kat', '00e90172', '08_kavya', ..., 'zig_zag_zig_',
       'zmorris1986', 'zoestarz16'], dtype='<U20')

In [173]:
unique_users_tweets

array(['00001Kat', '00e90172', '08_kavya', ..., 'zig_zag_zig_',
       'zmorris1986', 'zoestarz16'], dtype=object)

In [174]:
missing_user = []

for i in unique_users_network:
    if i not in unique_users_tweets:
        missing_user.append(i)

In [175]:
len(unique_users_network)

5322

In [176]:
len(missing_user)

2092

In [177]:
import snscrape.modules.twitter
import pandas as pd

info2 = []
for name in missing_user:
    try:
        scraper = snscrape.modules.twitter.TwitterUserScraper(name)
        if scraper.entity:
            info2.append([name, scraper.entity.followersCount])
    except:
        pass 

In [178]:
more = pd.DataFrame(info2, columns =['Username', 'followersCount'])

In [179]:
more

,Username,followersCount
0,100BlackMen,18201
1,1423Brown,3098
2,1GunnaGunna,2288597
3,1Mauricewood,4231
4,1TurboTheGreat,20031
...,...,...
1762,z_hurston,13
1763,zahranc,5471
1764,zazzybritches,15734
1765,zdiniJAMA,271


In [180]:
# merging dataset 
user_data = tweets.drop('Text',1).drop_duplicates(subset='Username', keep="last").append(more).reset_index(drop=True)

C:\Users\crist\AppData\Local\Temp/ipykernel_14796/2850779970.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  user_data = tweets.drop('Text',1).drop_duplicates(subset='Username', keep="last").append(more).reset_index(drop=True)
C:\Users\crist\AppData\Local\Temp/ipykernel_14796/2850779970.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  user_data = tweets.drop('Text',1).drop_duplicates(subset='Username', keep="last").append(more).reset_index(drop=True)


In [181]:
user_data # complete database of all users that have been mentionned/have tweeted

,Username,user_mention,followersCount
0,ScribeShelly,NaN,1971
1,NobleJennJenn,NaN,1115
2,gndinaa,NaN,709
3,revfredsmall,"AttorneyCrump, mlevenson",2903
4,p_tomblin,NaN,38
...,...,...,...
5012,z_hurston,NaN,13
5013,zahranc,NaN,5471
5014,zazzybritches,NaN,15734
5015,zdiniJAMA,NaN,271


In [182]:
top100 = user_data.nlargest(100,'followersCount')
users = top100.Username 

In [183]:
users

4405      barackobama
3363      BarackObama
4329          YouTube
3786    KimKardashian
4962          twitter
            ...      
4306        WORLDSTAR
4876      seanhannity
4478           common
4166          Spotify
3856          MTVNEWS
Name: Username, Length: 100, dtype: object

In [184]:
network3 = network2[(network2.source.isin(users)) | (network2['target'].isin(users))]

In [185]:
network3=network3.drop(columns="followersCount")

In [186]:
network3.to_csv("networkx.csv") # outputting the network to use for closeness, betweeness, and degree